In [1]:
# -*- coding: utf-8 -*-
"""lime_shap_st_multiplicative_norm_zero.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Rjw096Slt1atIGGrFC_G3wU8ivBfmIoF

### ***Cineca***
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
from keras import activations
import numpy as np
import pickle
import os
from keras.models import load_model
from skimage.transform import resize
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
import pickle
import numpy as np
import geopandas as gpd
import xarray
import rioxarray
from skimage.segmentation import slic
import matplotlib.pyplot as plt


2025-03-05 17:46:06.729510: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 17:46:09.194569: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 17:46:17.026225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 17:46:17.030177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 17:46:17.856718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:

# IMPORTO I DATI PER VOTTIGNASCO
import os

# Ottieni il percorso effettivo da una variabile d'ambiente
work_path = os.environ['WORK']  # Ottieni il valore della variabile d'ambiente WORK
v_test_OHE_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_month_OHE.npy")
v_test_image_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_normalized_image_sequences.npy")
v_test_target_dates_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_target_dates.npy")
v_test_images_dates_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_image_sequences_dates.npy")
shapefile_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/shapefile_raster/")

# Carica l'array numpy dai file
vottignasco_test_OHE    = np.load(v_test_OHE_path)
vottignasco_test_image  = np.load(v_test_image_path)
vottignasco_test_dates  = np.load(v_test_target_dates_path)
vottignasco_test_images_dates = np.load(v_test_images_dates_path)

print(len(vottignasco_test_dates))
print(len(vottignasco_test_image))
print(len(vottignasco_test_OHE))


# """##### ***Black Boxes***"""

import os
import tensorflow as tf
from keras.models import load_model

# Se vuoi abilitare il dropout a runtime
mc_dropout = True

# Definizione della classe personalizzata doprout_custom
class doprout_custom(tf.keras.layers.SpatialDropout1D):
    def call(self, inputs, training=None):
        if mc_dropout:
            return super().call(inputs, training=True)
        else:
            return super().call(inputs, training=False)

# Percorso della directory su Cineca
base_dir = os.path.join(os.environ['WORK'], "Water_Resources/rise-video/trained_models/seq2val/Vottignasco")
lstm_suffix = 'time_dist_LSTM'

vott_lstm_models = []

def extract_index(filename):
    """Funzione per estrarre l'indice finale dal nome del file."""
    return int(filename.split('_LSTM_')[-1].split('.')[0])

# Trova tutti i file .keras nella cartella e li aggiunge alla lista
for filename in os.listdir(base_dir):
    if lstm_suffix in filename and filename.endswith(".keras"):
        vott_lstm_models.append(os.path.join(base_dir, filename))

# Ordina i modelli in base all'indice finale
vott_lstm_models = sorted(vott_lstm_models, key=lambda x: extract_index(os.path.basename(x)))

# Lista per i modelli caricati
vott_lstm_models_loaded = []

for i, model_lstm_path in enumerate(vott_lstm_models[:10]):  # Prendo i primi 10 modelli ordinati
    #print(f"Caricamento del modello LSTM {i+1}: {model_lstm_path}")

    # Carico il modello con la classe custom
    model = load_model(model_lstm_path, custom_objects={"doprout_custom": doprout_custom})

    # Aggiungo il modello alla lista
    vott_lstm_models_loaded.append(model)

print(vott_lstm_models_loaded)


"""#### ***Application Masks***"""

def multiplicative_uniform_noise_onechannel(instance, zs_primes, masks, channel, std_zero_value=-0.6486319166678826):
  """
  param:masks: maschere generate per ogni superpixel
  """
  masked = []
  for z in zs_primes:
    masked_instance = copy.deepcopy(instance)
    for i,z_i in enumerate(z):
      if z_i == 0:
         # Applica la perturbazione solo al canale specificato
        masked_instance[..., channel] = (
            masked_instance[..., channel] * masks[i] + (1 - masks[i]) * std_zero_value)

    masked.append(masked_instance)

  return masked

"""#### ***Predizione sulle Istanze Perturbate***"""

import tensorflow as tf
import numpy as np

def ensemble_predict(models, images, x3_exp, batch_size=1000):
    # Assicuriamoci che images sia una lista
    if not isinstance(images, list):
        images = [images]

    len_x3 = len(images)

    # Convertiamo x3_exp in un tensore replicato per ogni immagine
    x3_exp_tensor = tf.convert_to_tensor(x3_exp, dtype=tf.float32)

    # Lista per raccogliere le predizioni finali
    final_preds = []

    # Processamento a batch
    for i in range(0, len_x3, batch_size):
        batch_images = images[i:i + batch_size]
        batch_len = len(batch_images)

        # Conversione batch in tensori
        Y_test = tf.stack([tf.convert_to_tensor(img, dtype=tf.float32) for img in batch_images])
        Y_test_x3 = tf.tile(tf.expand_dims(x3_exp_tensor, axis=0), [batch_len, 1, 1])

        # Raccoglie le predizioni di tutti i modelli per il batch corrente
        batch_preds = []

        for model in models:
            preds = model.predict([Y_test, Y_test_x3], verbose=0)
            batch_preds.append(preds)

        # Converte le predizioni del batch in un tensore e calcola la media
        batch_preds_tensor = tf.stack(batch_preds)
        mean_batch_preds = tf.reduce_mean(batch_preds_tensor, axis=0)

        # Aggiunge le predizioni del batch alla lista finale
        final_preds.extend(mean_batch_preds.numpy())

    return np.array(final_preds)

"""###### ***Spatial-Superpixels***"""

import geopandas as gpd
import xarray
import rioxarray
from skimage.segmentation import slic
import matplotlib.pyplot as plt

def create_spatial_superpixels(shapefile_path, n_segments=8, compactness=15):
  # DTM [50m] import
  dtm_piemonte = rioxarray.open_rasterio(shapefile_path + 'DTMPiemonte_filled_50m.tif')
  dtm_piemonte = dtm_piemonte.rio.reproject("epsg:4326")
  dtm_piemonte = dtm_piemonte.where(dtm_piemonte != -99999) # Take valid pixel

  # Catchment shapefile
  catchment = gpd.read_file(shapefile_path + "BAC_01_bacialti.shp") # select GRANA-MAIRA	and VARAITA
  catchment = catchment.to_crs('epsg:4326')

  # Select only the Grana-Maira catchment
  catchment_GM = catchment.loc[catchment.NOME == "GRANA-MAIRA"]
  catchment_GM = catchment_GM.reset_index(drop = True)

  # Retrieve the borders of the catchment from the shapefile
  xmin_clip, ymin_clip, xmax_clip, ymax_clip = catchment_GM.total_bounds
  # Extend the borders to include more pixel on the borders

  increase = 0.05 # Degrees
  #ymin_clip -= increase # not needed
  xmin_clip += increase # "+" for subset for pixel included in the mask
  xmax_clip += increase
  #ymax_clip += increase # not needed

  dtm_piemonte_clipped = dtm_piemonte.rio.clip_box( minx = xmin_clip, maxx= xmax_clip , miny= ymin_clip , maxy= ymax_clip)

  # Creazione img 5x8 cone lat,lon,dtm
  # Definizione delle coordinate
  lon = np.array([6.938, 7.063, 7.188, 7.313, 7.438, 7.563, 7.688, 7.813])  # 8 valori
  lat = np.array([44.313, 44.438, 44.563, 44.688, 44.813])  # 5 valori

  # Creazione di una griglia lat-lon 5x8
  lon_grid, lat_grid = np.meshgrid(lon, lat)

  # Creazione di un array 5x8x3
  img = np.zeros((5, 8, 3))

  # Assegno le coordinate nei primi due canali
  img[:, :, 0] = lat_grid  # Canale 0 = latitudine
  img[:, :, 1] = lon_grid  # Canale 1 = longitudine
  img[:, :, 2] = 0  # Canale 2 = valore placeholder

  for nr_lat,latitude in enumerate(lat):
    for  nr_lon,longitude in enumerate(lon):
      img[nr_lat, nr_lon, 2] = dtm_piemonte_clipped.sel(x=longitude, y=latitude, method='nearest').values

  img = np.nan_to_num(img, nan=0.0)

  # SLIC
  segments = slic(img, n_segments=n_segments, compactness=compactness)

  # Creazione Spatial-Superpixels
  # Trova i valori unici nella matrice (i cluster)
  clusters = np.unique(segments)

  # Creazione di una lista di matrici binarie per ogni cluster
  binary_matrices = {}

  for cluster in clusters:
      binary_matrices[cluster] = (segments == cluster).astype(int)

  spatial_superpixels = [matrix for _, matrix in binary_matrices.items()]

  spatial_superpixel_clusters = []

  for ss in spatial_superpixels:
    indices = np.argwhere(ss == 1)
    cluster_pixels = [(x, y) for x, y in indices]
    spatial_superpixel_clusters.append(cluster_pixels)

  return spatial_superpixels, spatial_superpixel_clusters, segments

105
105
105
[<keras.src.engine.functional.Functional object at 0x7fb86b4ed690>, <keras.src.engine.functional.Functional object at 0x7fb86adfcc10>, <keras.src.engine.functional.Functional object at 0x7fb86abcf790>, <keras.src.engine.functional.Functional object at 0x7fb86ac62cb0>, <keras.src.engine.functional.Functional object at 0x7fb86aacf910>, <keras.src.engine.functional.Functional object at 0x7fb86ab91300>, <keras.src.engine.functional.Functional object at 0x7fb86aa2e020>, <keras.src.engine.functional.Functional object at 0x7fb86aa92bf0>, <keras.src.engine.functional.Functional object at 0x7fb86a9278b0>, <keras.src.engine.functional.Functional object at 0x7fb86a7abe80>]


In [ ]:

"""#### ***Evaluation Metrics***"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

def calculate_auc(x, y):
    """
    Calcola l'area sotto la curva (AUC) utilizzando il metodo del trapezio.

    :param x: Valori dell'asse x (frazione dei pixel/frame inseriti).
    :param y: Valori dell'asse y (errori calcolati).
    :return: Area sotto la curva.
    """
    return np.trapz(y, x)

def calculate_auc_and_mean_errors(errors_all_dateset):
  mean_errors = np.mean(errors_all_dateset, axis=0)
  # Array x per il numero di superpixel inseriti
  x = np.arange(0, len(mean_errors))  # Array dinamico basato sulla lunghezza dei dati
  auc = calculate_auc(x, mean_errors)

  return auc,mean_errors

"""##### ***Insertion***"""

def update_instance_with_superpixels(current_instance, original_instance, start, end, list_of_pixel):
    """
    Aggiorna l'immagine inserendo i pixel più importanti.

    :param current_instance: Istanza corrente.
    :param original_instance: Istanza originale.
    :param index_of_superpixels: Lista contente gli indici del superpixel considerato
    :return: Istanza aggiornata con il superpixel.
    """
    new_current_instance = current_instance.copy()

    for x,y in list_of_pixel:
      for t in range(start,end):
        new_current_instance[t, x, y, :] = original_instance[t, x, y, :]
    return new_current_instance

def insertion(models, original_instance, x3, sorted_per_importance_all_superpixels_index, initial_blurred_instance, original_prediction):
    """
    Calcola la metrica di inserimento per una spiegazione data.

    :param models: Lista di modelli pre-addestrati.
    :param original_instance: Istanza originale.
    :param x3: Codifica one-hot per la previsione.
    :param sorted_per_importance_all_superpixels_index: Lista di liste di tutti i superpixel per importanza
    :param initial_blurred_images: Immagine iniziale con tutti i pixel a zero.
    :return: Lista degli errori ad ogni passo di inserimento.
    """

    # Lista per memorizzare le istanze a cui aggiungo pixel mano a mano. Inizializzata con istanza iniziale blurrata
    insertion_images = [initial_blurred_instance]

    # Predizione sull'immagine iniziale (tutti i pixel a zero)
    I_prime = copy.deepcopy(initial_blurred_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel aggiunti in maniera graduale
    for start,end,list_of_pixel in sorted_per_importance_all_superpixels_index:
        I_prime = update_instance_with_superpixels(I_prime, original_instance, start,end,list_of_pixel)
        insertion_images.append(I_prime)

    # Calcolo le predizioni sulle istanze a cui ho aggiunto i pixel in maniera graduale
    new_predictions = ensemble_predict(models, insertion_images, x3)
    # Rispetto ad ogni suddetta predizione, calcolo il MSE rispetto la pred sull'istanza originaria (come da test-set). Ignora la prima che è sull'img blurrata originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in new_predictions[1:]]

    initial_error = mean_squared_error(original_prediction, new_predictions[0])
    print(f"Initial Prediction with Blurred Instance, new prediction: {new_predictions[0]}, error: {initial_error}")
    only_inserted_pixel_new_predictions = new_predictions[1:]

    for nr_superpixel, error in enumerate(errors):
      print(f"SuperPixel: {sorted_per_importance_all_superpixels_index[nr_superpixel]}, new prediction: {only_inserted_pixel_new_predictions[nr_superpixel]}, error: {error}")

    total_errors = [initial_error] + errors # Errore iniziale + errori su tutti i pixel inseriti

    # Nuovo asse X: numero di superpixel inseriti (1, 2, ..., 8)
    x = np.arange(0, len(total_errors))  # Da 0 a 8 inclusi
    #print(x)

    x_for_auc = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC con il nuovo asse x
    auc = calculate_auc(x_for_auc, total_errors)
    print(f"Area under the curve (AUC): {auc}")

    # # Plot della curva dell'errore e area sotto la curva (AUC)
    # plt.plot(x, total_errors, linestyle='-', label='Error curve', color='blue')
    # # Pallini blu sui punti della curva
    # #plt.scatter(x, total_errors, color='blue', zorder=3)

    # # Area sotto la curva
    # plt.fill_between(x, total_errors, color='skyblue', alpha=0.4)

    # # Testo AUC in alto a destra
    # plt.text(x[-1] * 0.95, max(total_errors) * 0.9, f'AUC: {auc:.2f}',
    #      horizontalalignment='right')

    # plt.xlabel('Number of superpixels inserted')  # Modifica etichetta asse X
    # plt.ylabel('Mean Squared Error')
    # plt.title('Insertion Metric Curve')
    # #plt.xticks(x)  # Imposta i tick esattamente sui numeri interi (1, 2, ..., 8)
    # plt.legend()
    # #plt.grid(True, linestyle='--', alpha=0.6)  # Griglia più leggibile
    # plt.show()
    return total_errors,auc

"""##### ***Deletion***"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

def update_instance_removing_superpixels(current_instance, start, end, list_of_pixel, std_zero_value=-0.6486319166678826):
    """
    Aggiorna l'immagine inserendo i pixel più importanti.

    :param current_instance: Istanza corrente.
    :param original_instance: Istanza originale.
    :param index_of_superpixels: Lista contente gli indici del superpixel considerato
    :return: Istanza aggiornata con il superpixel.
    """
    new_current_instance = current_instance.copy()
    for x,y in list_of_pixel:
      for t in range(start,end):
        new_current_instance[t, x, y, 0] = std_zero_value
        new_current_instance[t, x, y, 1] = 0.0
        new_current_instance[t, x, y, 2] = 0.0
    return new_current_instance

def deletion(models, original_instance, x3_instance, sorted_per_importance_all_superpixels_index, original_prediction):
    """
    Calcola la metrica di deletion per una spiegazione data.

    :param models: Lista di modelli pre-addestrati.
    :param original_instance: Istanza originale.
    :param x3_instance: Codifica one-hot per la previsione.
    :param sorted_per_importance_all_superpixels_index: Lista di liste di tutti i superpixel per importanza
    :param original_prediction: Predizione originale.
    :return: Lista degli errori ad ogni passo di deletion.
    """

    # Lista per memorizzare le istanze a cui aggiungo pixel mano a mano. Inizializzata con istanza originale
    deletion_images = []

    # Predizione sull'immagine iniziale (tutti i pixel a zero)
    I_prime = copy.deepcopy(original_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel aggiunti in maniera graduale
    for start,end,list_of_pixel in sorted_per_importance_all_superpixels_index:
        I_prime = update_instance_removing_superpixels(I_prime, start, end, list_of_pixel)
        deletion_images.append(I_prime)

    # Calcolo della predizione su tutte le img a cui ho rimosso gradualmente i pixel
    new_predictions = ensemble_predict(models, deletion_images, x3_instance)
    # Calcolo del mse rispetto la predizione originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in new_predictions]

    initial_error = 0.0
    print(f"Initial Prediction with Original instance, prediction: {original_prediction}, error: {initial_error}")

    for nr_superpixel, error in enumerate(errors):
      print(f"Removed SuperPixel: {sorted_per_importance_all_superpixels_index[nr_superpixel]}, new prediction: {new_predictions[nr_superpixel]}, error: {error}")

    total_errors = [initial_error] + errors # Errore iniziale + errori su tutti i pixel rimossi

    # Plot
    # Nuovo asse X: numero di superpixel inseriti (1, 2, ..., 8)
    x = np.arange(0, len(total_errors))  # Da 0 a 8 inclusi
    #print(x)
    x_for_auc = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC con il nuovo asse x
    auc = calculate_auc(x_for_auc, total_errors)
    print(f"Area under the curve (AUC): {auc}")

    # # Plot della curva dell'errore e area sotto la curva (AUC)
    # plt.plot(x, total_errors, linestyle='-', label='Error curve', color='blue')
    # # Area sotto la curva
    # plt.fill_between(x, total_errors, color='lightcoral', alpha=0.4)
    # # Testo "Error curve" in basso a destra
    # plt.legend(['Error curve'], loc='lower right', bbox_to_anchor=(0.97, 0.02))

    # # Testo AUC leggermente sotto la legenda
    # plt.text(0.941, 0.13, f'AUC: {auc:.2f}',
    #      transform=plt.gca().transAxes,
    #      fontsize=10,
    #      verticalalignment='bottom',
    #      horizontalalignment='right',
    #      bbox=dict(facecolor='white', alpha=0.6, edgecolor='grey'))

    # plt.xlabel('Number of superpixels removed')
    # plt.ylabel('Mean Squared Error')
    # plt.title('Deletion Metric Curve')

    # plt.show()
    return total_errors,auc

In [ ]:
def create_frame_for_saliency_video(shape, coefficients, spatial_superpixels, height=5,width=8):
  """
  Args:
   - shape: (time_steps, heigth, width, nr_channels)
   - coefficients: coefficienti spatial_superpixel per clustet temporale
   - spatial_superpixels: matrici 5x8 per i superpixels spaziali
  """

  frame = np.zeros((height, width))

  for i,superpixel in enumerate(spatial_superpixels):
    frame += superpixel * coefficients[i]

  return frame

def find_top_indices(matrix):
    # Flatten della matrice e ordinamento degli indici in base ai valori decrescenti
    flat_indices = np.argsort(matrix.flatten())[::-1]
    # Converti gli indici "piatti" in coordinate (x, y)
    indices = [np.unravel_index(idx, matrix.shape) for idx in flat_indices]
    return indices

def calculate_saliency_video_insertion_deletion_errors_auc(nr_instance, abs_coefficients, temporal_superpixels, spatial_superpixels, spatial_superpixel_clusters,
                                                           nr_temporal_superpixel, nr_spatial_superpixel,
                                                           models, channel_prec=0, std_zero_value=-0.6486319166678826,input_size=(104,5,8,3),T=104,H=5,W=8,C=3):
  abs_coefficients_reshape = abs_coefficients.reshape(nr_temporal_superpixel, nr_spatial_superpixel)

  saliency_video = np.zeros((T, H, W))

  # Frame per ogni stagione individuata
  frames_for_t_superpixels = [create_frame_for_saliency_video(input_size, coeff, spatial_superpixels) for coeff in abs_coefficients_reshape]

  for i,t_superpixel in enumerate(temporal_superpixels):
      start, end, _ = t_superpixel
      saliency_video[start:end+1] = frames_for_t_superpixels[i]

  # ST-Superpixels ordinati per importanza
  superpixels_importance_cluster = find_top_indices(abs_coefficients_reshape)
  sorted_per_importance_all_superpixels_index = []
  for nr_ts, nr_ss in superpixels_importance_cluster:
    start,end = temporal_superpixels[nr_ts][0], temporal_superpixels[nr_ts][1] + 1
    cluster_spatial_superpixel = spatial_superpixel_clusters[nr_ss]
    sorted_per_importance_all_superpixels_index.append((start, end, cluster_spatial_superpixel))

  # Insertion
  instance    = copy.deepcopy(vottignasco_test_image[nr_instance])
  x3_instance = copy.deepcopy(vottignasco_test_OHE[nr_instance])    # One-Hot encode mesi dei frame dell'istanza

  all_superpixels_index = sorted_per_importance_all_superpixels_index
  initial_blurred_instance = np.zeros((T,H,W,C))
  initial_blurred_instance[:,:,:,channel_prec] = std_zero_value

  original_prediction = ensemble_predict(models, instance, x3_instance)

  errors_insertion,auc_insertion = insertion(models, instance, x3_instance, all_superpixels_index, initial_blurred_instance, original_prediction)

  # Deletion
  errors_deletion,auc_deletion = deletion(models, instance, x3_instance, all_superpixels_index, original_prediction)

  return errors_insertion,auc_insertion,errors_deletion,auc_deletion

In [3]:
import pickle

#results_setups_path = "./MyDrive/Water_Resources/wr_extensions/superpixels_versions/LIME/last_try/Temporal/results/lime_temporal_insertion_deletion_results_setup_kw_80_alpha_0.pkl"
path_to_load_results = os.path.join(work_path, "Water_Resources/rise-video/XAI/spatial_temporal/results/lime_shap_multiplicative_norm_zero/")
results_string_name  = f"lime_shap_st_results_setup_ns_7_comp_10.pkl"

# Apri il file in modalità lettura binaria ('rb')
with open(path_to_load_results + results_string_name, 'rb') as file:
    lime_shap_st_results = pickle.load(file)

In [8]:
lime_shap_st_results["shap"]["ns_7_comp_10"]["coefficients"][0]

array([-0.0030276 ,  0.10265689, -0.02457111, -0.02044141,  0.05291649,
        0.05305451, -0.04245743, -0.07580339, -0.06894456, -0.07425273,
        0.01262863,  0.02167514,  0.03006815,  0.03237063, -0.01814097,
       -0.04687366, -0.04395362, -0.00689713,  0.00914542,  0.00138899,
       -0.07734996, -0.06819287,  0.0461255 ,  0.01255203,  0.05487231,
        0.00267578,  0.01659084, -0.03199145, -0.02128805, -0.04388248,
       -0.12688617, -0.04007857, -0.01024403, -0.01708648, -0.0364305 ,
       -0.01811585, -0.01398087,  0.04439715, -0.08252574, -0.01017479,
        0.01479222,  0.04346734,  0.06472448,  0.00404835,  0.04931687,
       -0.00191776, -0.01297372, -0.02926882,  0.00441794, -0.60417607,
       -0.20549155, -0.1443521 , -0.36567363, -0.09242635, -0.5516351 ,
       -0.10727446, -0.01366569,  0.05735828,  0.08363638, -0.0571634 ,
        0.07286287,  0.07203234,  0.0464308 ])

In [ ]:
for nr_instance in range(len(vottignasco_test_image)):
    coefficient_i = lime_shap_st_results["shap"]["ns_7_comp_10"]["coefficients"][nr_instance]

    